Stap 0 – spaCy installeren + model downloaden

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Stap 1 – Laad de tekst

In [ ]:
#Text importeren en opslaan in een variable "text"

text = open("Original.txt", "r", encoding="utf-8").read()
print(text)



Artificial intelligence (AI) has evolved from a niche research topic into one of the most transformative forces of the 21st century. While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks, and large-scale data processing have dramatically expanded the possibilities of what AI can accomplish. Today, AI technologies influence nearly every sector of society, including healthcare, finance, education, entertainment, manufacturing, and public administration. The pace of this evolution has brought both excitement and uncertainty, prompting ongoing debates about how AI should be developed, deployed, and regulated.

One of the most significant areas where AI has made an impact is healthcare. Machine learning models are increasingly used to analyze medical images, detect patterns in large datasets, and assist doctors in diagnosing diseases at earlier stages. For example, AI-driven diagnostic tools can identify abnor

In [ ]:
# Lengte van de tekst bekijken
print("Aantal karakters:", len(text))
print("Aantal woorden:", len(text.split()))

# Hele tekst laten zien (kan lang zijn!)
print("\n--- ORIGINELE TEKST ---\n")
print(text)


Aantal karakters: 7898
Aantal woorden: 1086

--- ORIGINELE TEKST ---

Artificial intelligence (AI) has evolved from a niche research topic into one of the most transformative forces of the 21st century. While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks, and large-scale data processing have dramatically expanded the possibilities of what AI can accomplish. Today, AI technologies influence nearly every sector of society, including healthcare, finance, education, entertainment, manufacturing, and public administration. The pace of this evolution has brought both excitement and uncertainty, prompting ongoing debates about how AI should be developed, deployed, and regulated.

One of the most significant areas where AI has made an impact is healthcare. Machine learning models are increasingly used to analyze medical images, detect patterns in large datasets, and assist doctors in diagnosing diseases at earl

Stap 2 – Preprocessing met spaCy

In [ ]:
import spacy

nlp = spacy.load("en_core_web_lg")

# Verwerk de tekst met spaCy
doc = nlp(text)

# Zinnen extraheren
sentences = [sent.text.strip() for sent in doc.sents]
print("Aantal zinnen:", len(sentences))
print("\nVoorbeeldzinnen:\n")
for s in sentences[:3]:
    print("-", s)


Aantal zinnen: 57

Voorbeeldzinnen:

- Artificial intelligence (AI) has evolved from a niche research topic into one of the most transformative forces of the 21st century.
- While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks, and large-scale data processing have dramatically expanded the possibilities of what AI can accomplish.
- Today, AI technologies influence nearly every sector of society, including healthcare, finance, education, entertainment, manufacturing, and public administration.


In [ ]:
def preprocess_with_spacy(sentence):
    # We sturen de zin door spaCy om hem te analyseren.
    # .lower() zorgt ervoor dat alle tekst naar kleine letters gaat,
    # zodat woorden als "Studenten" en "studenten" gelijk worden behandeld.
    doc = nlp(sentence.lower())

    # We bouwen een lijst met tokens (woorden) die we willen behouden.
    # We filteren:
    # - token.is_stop → stopwoorden zoals "de", "het", "and", "the" (weinig informatief)
    # - token.is_punct → leestekens zoals punt, komma, etc.
    tokens = [token.text for token in doc
              if not token.is_stop and not token.is_punct]

    # We geven de lijst met opgeschoonde woorden terug.
    return tokens


# We voeren de functie uit voor elke zin in onze tekst.
# processed_sentences wordt dus een lijst van lijsten:
# elke binnenste lijst bevat de belangrijke woorden van één zin.
processed_sentences = [preprocess_with_spacy(s) for s in sentences]

print("\nEerste 3 verwerkte zinnen:\n")

# We printen de eerste drie verwerkte zinnen
# zodat je kunt zien hoe spaCy de tekst heeft opgeschoond.
for s in processed_sentences[:3]:
    print(s)



Eerste 3 verwerkte zinnen:

['artificial', 'intelligence', 'ai', 'evolved', 'niche', 'research', 'topic', 'transformative', 'forces', '21st', 'century']
['early', 'ai', 'systems', 'limited', 'narrow', 'problem', 'solving', 'tasks', 'recent', 'developments', 'machine', 'learning', 'deep', 'neural', 'networks', 'large', 'scale', 'data', 'processing', 'dramatically', 'expanded', 'possibilities', 'ai', 'accomplish']
['today', 'ai', 'technologies', 'influence', 'nearly', 'sector', 'society', 'including', 'healthcare', 'finance', 'education', 'entertainment', 'manufacturing', 'public', 'administration']


Stap 3A – TF-IDF Sentence Scoring (met spaCy-splitsing)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# We maken een TF-IDF vectorizer aan.
# 'stop_words="english"' betekent dat veelvoorkomende Engelse woorden (the, and, of)
# worden genegeerd om de belangrijkste woorden over te houden.
vectorizer_tfidf = TfidfVectorizer(stop_words="english")

# We zetten alle zinnen om in een TF-IDF matrix.
# Elke rij = één zin.
# Elke kolom = één woord dat voorkomt in de tekst.
# Elke cel = TF-IDF score van dat woord in die zin.
tfidf_matrix = vectorizer_tfidf.fit_transform(sentences)

# We berekenen de score voor elke zin door alle TF-IDF scores in die zin op te tellen.
# Hoe hoger de som → hoe meer 'informatieve' woorden → hoe belangrijker de zin.
sentence_scores = tfidf_matrix.sum(axis=1).A1

# We sorteren de zinnen op basis van hun score.
# np.argsort geeft de indices van klein → groot.
# [::-1] draait de lijst om, zodat we groot → klein krijgen.
sorted_idx = np.argsort(sentence_scores)[::-1]

# Hoeveel zinnen willen we in de samenvatting opnemen?
# Dit kun je aanpassen aan wat je nodig vindt.
top_n = 5

# We kiezen de top-n zinnen (die met de hoogste TF-IDF score).
summary_tfidf = [sentences[i] for i in sorted_idx[:top_n]]

print("=== TF-IDF SUMMARY ===\n")

# We printen de daadwerkelijke TF-IDF samenvatting.
for s in summary_tfidf:
    print("-", s)


=== TF-IDF SUMMARY ===

- While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks, and large-scale data processing have dramatically expanded the possibilities of what AI can accomplish.
- Governments and private companies alike recognize the potential of AI to stimulate economic growth, drive innovation, and address complex global issues such as climate change, cybersecurity, and public health.
- For example, AI-driven diagnostic tools can identify abnormalities in X-rays or MRI scans more quickly than human specialists, particularly in regions facing shortages of trained radiologists.
- Machine learning models are increasingly used to analyze medical images, detect patterns in large datasets, and assist doctors in diagnosing diseases at earlier stages.
- For example, facial recognition systems have been shown to perform less accurately on individuals with darker skin tones, raising serious concerns about 

In [ ]:
print(summary_tfidf)


['While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks, and large-scale data processing have dramatically expanded the possibilities of what AI can accomplish.', 'Governments and private companies alike recognize the potential of AI to stimulate economic growth, drive innovation, and address complex global issues such as climate change, cybersecurity, and public health.', 'For example, AI-driven diagnostic tools can identify abnormalities in X-rays or MRI scans more quickly than human specialists, particularly in regions facing shortages of trained radiologists.', 'Machine learning models are increasingly used to analyze medical images, detect patterns in large datasets, and assist doctors in diagnosing diseases at earlier stages.', 'For example, facial recognition systems have been shown to perform less accurately on individuals with darker skin tones, raising serious concerns about discrimination and m

In [ ]:
print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 712 stored elements and shape (57, 479)>
  Coords	Values
  (0, 33)	0.29266984696268117
  (0, 245)	0.29266984696268117
  (0, 18)	0.12015015412517116
  (0, 159)	0.3226225177023495
  (0, 285)	0.3226225177023495
  (0, 368)	0.29266984696268117
  (0, 443)	0.3226225177023495
  (0, 453)	0.3226225177023495
  (0, 187)	0.3226225177023495
  (0, 0)	0.3226225177023495
  (0, 65)	0.3226225177023495
  (1, 18)	0.16754704396449846
  (1, 134)	0.2249454008236202
  (1, 428)	0.13265761733580683
  (1, 262)	0.2249454008236202
  (1, 278)	0.2249454008236202
  (1, 324)	0.2249454008236202
  (1, 404)	0.2249454008236202
  (1, 429)	0.2249454008236202
  (1, 350)	0.2249454008236202
  (1, 116)	0.2249454008236202
  (1, 263)	0.189243612707906
  (1, 256)	0.1604195983464265
  (1, 106)	0.2249454008236202
  (1, 284)	0.2249454008236202
  :	:
  (54, 242)	0.3093834692131872
  (55, 219)	0.2853663166402481
  (55, 177)	0.3629992331220156
  (55, 340)	0.2994902362346731
  

Stap 3B – TextRank (met spaCy-zinnen)

In [ ]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

# We maken opnieuw een TF-IDF vectorizer aan,
# maar nu met Engelse stopwoorden eruit gefilterd
vectorizer = TfidfVectorizer(stop_words="english")

# We zetten alle zinnen om in een TF-IDF matrix
# Elke rij = een zin, elke kolom = een woord, elke cel = TF-IDF gewicht
X = vectorizer.fit_transform(sentences)

# We berekenen hoe sterk elke zin lijkt op elke andere zin
# Dit gebeurt met cosine similarity (hoeveel overlap in woordgebruik)
similarity_matrix = cosine_similarity(X)

# We zetten de diagonalen op 0,
# omdat een zin niet vergeleken moet worden met zichzelf
np.fill_diagonal(similarity_matrix, 0)

# We zetten de similarity matrix om in een graaf (network graph)
# Elke zin wordt een knoop (node), elke similarity een verbinding (edge)
graph = nx.from_numpy_array(similarity_matrix)

# We passen PageRank toe op de graaf
# Zinnen die verbonden zijn met veel andere belangrijke zinnen
# krijgen automatisch een hogere score
scores = nx.pagerank(graph)

# We sorteren alle zinnen op basis van hun PageRank score (hoog → laag)
ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# We kiezen de indices van de top-zinnen (top_n komt uit je instelling)
top_indices = [idx for idx, score in ranked[:top_n]]

# We sorteren de gekozen indices zodat de samenvatting in logische volgorde staat
top_indices_sorted = sorted(top_indices)

# We halen de bijbehorende zinnen op als onze TextRank samenvatting
summary_textrank = [sentences[i] for i in top_indices_sorted]

print("=== TEXTRANK SUMMARY ===\n")

# En we printen de uiteindelijke samenvatting, zin per zin
for s in summary_textrank:
    print("-", s)


=== TEXTRANK SUMMARY ===

- While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks, and large-scale data processing have dramatically expanded the possibilities of what AI can accomplish.
- Concerns about data privacy, algorithmic transparency, and the risk of over-reliance on automated systems continue to shape the conversation around AI-based medical tools.
- Critics argue that AI systems can perpetuate or even amplify existing societal inequalities if they are trained on biased data or deployed without sufficient oversight.
- AI systems often require vast amounts of information to function effectively, raising questions about how data is collected, who has access to it, and how it is protected.
- Despite the challenges, investment in AI continues to grow.


Stap 4 – Toon alles naast elkaar

In [ ]:
print("=== ORIGINAL TEXT (preview) ===\n")
print(text[:800], "...\n")

print("=== TF-IDF SUMMARY ===\n")
for s in summary_tfidf:
    print("-", s)

print("\n=== TEXTRANK SUMMARY ===\n")
for s in summary_textrank:
    print("-", s)


=== ORIGINAL TEXT (preview) ===

Artificial intelligence (AI) has evolved from a niche research topic into one of the most transformative forces of the 21st century. While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks, and large-scale data processing have dramatically expanded the possibilities of what AI can accomplish. Today, AI technologies influence nearly every sector of society, including healthcare, finance, education, entertainment, manufacturing, and public administration. The pace of this evolution has brought both excitement and uncertainty, prompting ongoing debates about how AI should be developed, deployed, and regulated.

One of the most significant areas where AI has made an impact is healthcare. Machine learning models are ...

=== TF-IDF SUMMARY ===

- While early AI systems were limited to very narrow problem-solving tasks, recent developments in machine learning, deep neural networks

# Reflectie

## TF-IDF Samenvatting
- Kiest zinnen met de meeste **belangrijke woorden**.  
- Pakt vaak feitelijke zinnen.  
- De samenvatting kan soms een beetje rommelig aanvoelen,  
  omdat TF-IDF niet kijkt naar de **betekenis** of **samenhang**.  
- Handig als je vooral belangrijke woorden of termen wilt vinden.

## TextRank Samenvatting
- Kijkt welke zinnen **het meest lijken op andere zinnen**.  
- Daardoor klinkt de samenvatting meestal **logischer en vloeiender**.  
- Soms mist TextRank een belangrijke zin,  
  als die zin niet veel lijkt op de rest.  
- Handig als je een **mooie, leesbare** samenvatting wilt.

## Vragen om over na te denken
- Welke samenvatting vertelt het verhaal het beste?  
- Welke samenvatting leest fijner? Waarom?  
- Welke methode zou jij gebruiken voor je eindopdracht?  
  - **TF-IDF** → veel belangrijke woorden  
  - **TextRank** → logisch en duidelijk verhaal  
- Wanneer denk je dat TF-IDF beter werkt?  
- Wanneer werkt TextRank juist beter?






## Welke samenvatting vertelt het verhaal het beste?
De **TextRank-samenvatting** vertelt het verhaal het beste, omdat die de hoofdboodschap en de structuur van de tekst goed laat terugkomen. Het klinkt meer als één logisch geheel.

## Welke samenvatting leest fijner? Waarom?
De **TextRank-samenvatting** leest fijner.  
Dat komt omdat TextRank kijkt naar de samenhang tussen zinnen. Daardoor voelt de samenvatting natuurlijker en makkelijker te begrijpen.

## Welke methode zou jij gebruiken voor je eindopdracht?
Ik zou **TextRank** gebruiken, omdat het de tekst beter samenvat op een manier die logisch en duidelijk is. TF-IDF kiest vooral zinnen met veel belangrijke woorden, maar dat is niet altijd het beste voor de leesbaarheid.

## Wanneer denk je dat TF-IDF beter werkt?
TF-IDF werkt beter als je:
- vooral de **informatieve kernwoorden** wilt hebben,  
- teksten hebt die heel **feitelijk of technisch** zijn,  
- of als je snel wilt zien welke zinnen de meeste “belangrijke termen” bevatten.

## Wanneer werkt TextRank juist beter?
TextRank werkt beter als je:
- een **vloeiende, begrijpelijke samenvatting** wilt,  
- een tekst hebt met een duidelijk verhaal of uitleg,  
- of als de doelgroep snel de kern van de tekst moet snappen.

